In [1]:
import torch
import torchvision
import torch.nn as nn
import math
import torch.nn.functional as F
import numpy as np
from torch.utils.data import Dataset
import pandas as pd
from torch.utils.data import DataLoader
from torchvision import transforms
import cv2
import matplotlib.pyplot as plt

In [2]:
sz = (256, 192)

In [3]:
class CapDataset(Dataset):
    def __init__(self, annotation_file):
        self.annotations = pd.read_csv(annotation_file)
        self.transform = transforms.Compose([None])

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_vec = torch.tensor(self.annotations.iloc[index, 1:], dtype=torch.float)
        img = torch.reshape(img_vec,(1, sz[1], sz[0]))
        # img = torch.transpose(img,1,2)

        y_label = torch.tensor(self.annotations.iloc[index, 0],dtype=torch.long)
        # if (y_label <= 61 ):
        #     img = torch.transpose(img,1,2)

        return (img, y_label)

In [4]:
device = 'cuda'
batch_size = 32

In [5]:
dataset_test = CapDataset("output_test_large.csv")
test_loader = DataLoader(dataset=dataset_test, shuffle=True, batch_size=batch_size)

In [6]:
import torch
from model import SpinalVGG
# from segmentation import Segmentation
import cv2 as cv2
import numpy as np
import os

device = 'cuda'
model1 = SpinalVGG().to(device)
model1.load_state_dict(torch.load("model_large_rect_optimal.h5"))

<All keys matched successfully>

In [7]:
tp = tn = fp = fn = total1 = correct1 = 0

for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)


            outputs = model1(images)
            _, predicted = torch.max(outputs.data, 1)
            total1 += labels.size(0)
            correct1 += (predicted == labels).sum().item()
            tp += ((predicted == 1) & (labels == 1)).sum().item()
            tn += ((predicted == 0) & (labels == 0)).sum().item()
            fp += ((predicted == 1) & (labels == 0)).sum().item()
            fn += ((predicted == 0) & (labels == 1)).sum().item()

C:\Users\Asus\AppData\Local\Temp\ipykernel_21032\43906973.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  img_vec = torch.tensor(self.annotations.iloc[index, 1:], dtype=torch.float)


In [8]:
model_precision= tp/(tp+fp)
model_recall= tp/(tp+fn)
model_f= 2*model_precision*model_recall/(model_precision+model_recall)
print("acc", correct1 / total1)
print('Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (0.5, model_f, model_precision, model_recall))
print(tp,fp,fn,tn)

acc 0.9518072289156626
Threshold=0.500000, F-Score=0.949, Precision=0.940, Recall=0.958
297 19 13 335
